In [69]:
import psycopg2
# import time
# import threading
# اطلاعات اتصال به CockroachDB
DATABASE_URL = "postgresql://root@localhost:26257/messenger?sslmode=disable"

# تابع اجرای فایل SQL
def execute_sql_file(filename):
    try:
        # اتصال به دیتابیس
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()

        # خواندن فایل SQL
        with open(filename, "r", encoding="utf-8") as file:
            sql_script = file.read()
        
        # اجرای دستورات SQL
        cursor.execute(sql_script)
        conn.commit()
        print("✅ فایل SQL با موفقیت اجرا شد!")

        # نمایش نتایج برخی از کوئری‌ها
        queries = [
            "SELECT s.codes, COUNT(s.codes) AS usage_count FROM Sticker s WHERE s.user_id = ( SELECT u.user_id FROM Users u JOIN contact c ON u.user_id = c.user_id WHERE c.user_name = 'mohammad' LIMIT 1 ) GROUP BY s.codes ORDER BY usage_count DESC LIMIT 3;",
            "WITH Mohammad AS (SELECT c.user_id  FROM contact c WHERE c.user_name = 'mohammad' ),  Ali AS ( SELECT c1.user_id  FROM contact c1  WHERE c1.user_name = 'ali') SELECT c.phone_number, c.first_name, c.last_name  FROM contact c JOIN Mohammad m ON c.user_id = m.user_id UNION SELECT c.phone_number, c.first_name, c.last_name  FROM contact c JOIN Ali a ON c.user_id = a.user_id;",
            "SELECT g.name, COUNT(t.message_id) AS message_count FROM Groupss g JOIN text_message t ON t.se_and_re_chat_id = g.chat_id WHERE g.member_count > 10000 GROUP BY g.name ORDER BY message_count;",
            "SELECT g.name FROM Groupss g JOIN text_message t ON t.se_and_re_chat_id = g.chat_id WHERE t.msg_date >= NOW() - INTERVAL '10 days' GROUP BY g.name ORDER BY COUNT(t.message_id) DESC LIMIT 1;",
            "WITH user_membership AS ( SELECT user_id, COUNT(*) AS groups_count FROM Member GROUP BY user_id ORDER BY groups_count DESC LIMIT 1 ) SELECT SUM(f.size) AS total_file_size FROM File f WHERE f.user_id = (SELECT user_id FROM user_membership);",
            "SELECT c.user_name FROM contact c JOIN Users u ON c.user_id = u.user_id JOIN Member m ON m.user_id = u.user_id JOIN Groupss g ON g.chat_id = m.chat_id GROUP BY c.user_name HAVING COUNT(m.chat_id) <= 2;",
            "SELECT COALESCE(phone_number, 0) AS phone FROM contact WHERE ROUND(phone_number/10000000) = 0912 AND (phone_number%10) =8;",
            "SELECT c.user_name FROM contact c JOIN Users u ON c.user_id = u.user_id JOIN Member m ON m.user_id = u.user_id JOIN channel c1 ON c1.chat_id = m.chat_id GROUP BY c.user_name HAVING COUNT(m.chat_id) = 2;",
            "WITH user_ids AS ( SELECT u.user_id FROM Users u JOIN contact c ON u.user_id = c.user_id WHERE c.user_name IN ('hossein', 'ali')), common_groups AS ( SELECT m.chat_id FROM Member m JOIN user_ids u ON m.user_id = u.user_id GROUP BY m.chat_id HAVING COUNT(DISTINCT m.user_id) = 2 ) SELECT t.message_id, t.content, t.msg_date, t.user_id, t.se_and_re_chat_id AS group_id FROM text_message t JOIN common_groups cg ON t.se_and_re_chat_id = cg.chat_id ORDER BY t.msg_date;",
            "SELECT AVG(LENGTH(t.content)) AS average_characters FROM text_message t JOIN Bot b ON t.se_and_re_chat_id = b.chat_id WHERE b.name = 'first_bot';"
        ]
        
        for query in queries:
            cursor.execute(query)
            results = cursor.fetchall()
            print(f"\n🔹 نتیجه کوئری:\n{results}")

        # def execute_query(query):
        #     try:
        #         start_time = time.time()
        #         cursor.execute(query)
        #         results = cursor.fetchall()
        #         elapsed_time = time.time() - start_time
        #         print(f"\n🔹 نتیجه کوئری:\n{results}")
        #         print(f"✅ Query executed in {elapsed_time:.4f} seconds")
        #     except Exception as e:
        #         print(f"❌ Query failed: {e}")

        # # اجرای همزمان کوئری‌ها با چندین ترد
        # threads = []
        # for query in queries:
        #     thread = threading.Thread(target=execute_query, args=(query,))
        #     threads.append(thread)
        #     thread.start()

        # for thread in threads:
        #     thread.join()

        # بستن اتصال
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"❌ خطا در اجرای SQL: {e}")

# اجرای فایل SQL
execute_sql_file("database_script.sql")


✅ فایل SQL با موفقیت اجرا شد!

🔹 نتیجه کوئری:
[(2, 3), (3, 2), (1, 1)]

🔹 نتیجه کوئری:
[(Decimal('9891000001'), 'Mohammad', 'Ahmadi'), (Decimal('9891000004'), 'Mohammad', 'Ahmadi'), (Decimal('9891000002'), 'Ali', 'Rezayi'), (Decimal('9891000006'), 'Ali', 'Rezayi')]

🔹 نتیجه کوئری:
[('Group5', 1), ('Group3', 2), ('Group1', 3)]

🔹 نتیجه کوئری:
[('Group1',)]

🔹 نتیجه کوئری:
[(Decimal('400'),)]

🔹 نتیجه کوئری:
[('sara',), ('ali',), ('hossein',)]

🔹 نتیجه کوئری:
[(Decimal('9120000008'),)]

🔹 نتیجه کوئری:
[('ali',)]

🔹 نتیجه کوئری:
[(1006, 'Funny text', datetime.datetime(2025, 1, 22, 19, 37, 4, 619858), 105, 502), (1010, 'Hi everyone', datetime.datetime(2025, 1, 22, 19, 37, 4, 619858), 102, 502)]

🔹 نتیجه کوئری:
[(Decimal('10.500000000000000000'),)]


In [68]:
import psycopg2
import time
import threading
# اطلاعات اتصال به CockroachDB
DATABASE_URL = "postgresql://root@localhost:26257/messenger?sslmode=disable"

# تابع اجرای فایل SQL
def execute_sql_file(filename):
    try:
        # اتصال به دیتابیس
        conn = psycopg2.connect(DATABASE_URL)
        cursor = conn.cursor()

        # خواندن فایل SQL
        with open(filename, "r", encoding="utf-8") as file:
            sql_script = file.read()
        
        # اجرای دستورات SQL
        cursor.execute(sql_script)
        conn.commit()
        print("✅ فایل SQL با موفقیت اجرا شد!")

        # نمایش نتایج برخی از کوئری‌ها
        queries = [
            "SELECT s.codes, COUNT(s.codes) AS usage_count FROM Sticker s WHERE s.user_id = ( SELECT u.user_id FROM Users u JOIN contact c ON u.user_id = c.user_id WHERE c.user_name = 'mohammad' LIMIT 1 ) GROUP BY s.codes ORDER BY usage_count DESC LIMIT 3;",
            "WITH Mohammad AS (SELECT c.user_id  FROM contact c WHERE c.user_name = 'mohammad' ),  Ali AS ( SELECT c1.user_id  FROM contact c1  WHERE c1.user_name = 'ali') SELECT c.phone_number, c.first_name, c.last_name  FROM contact c JOIN Mohammad m ON c.user_id = m.user_id UNION SELECT c.phone_number, c.first_name, c.last_name  FROM contact c JOIN Ali a ON c.user_id = a.user_id;",
            "SELECT g.name, COUNT(t.message_id) AS message_count FROM Groupss g JOIN text_message t ON t.se_and_re_chat_id = g.chat_id WHERE g.member_count > 10000 GROUP BY g.name ORDER BY message_count;",
            "SELECT g.name FROM Groupss g JOIN text_message t ON t.se_and_re_chat_id = g.chat_id WHERE t.msg_date >= NOW() - INTERVAL '10 days' GROUP BY g.name ORDER BY COUNT(t.message_id) DESC LIMIT 1;",
            "WITH user_membership AS ( SELECT user_id, COUNT(*) AS groups_count FROM Member GROUP BY user_id ORDER BY groups_count DESC LIMIT 1 ) SELECT SUM(f.size) AS total_file_size FROM File f WHERE f.user_id = (SELECT user_id FROM user_membership);",
            "SELECT c.user_name FROM contact c JOIN Users u ON c.user_id = u.user_id JOIN Member m ON m.user_id = u.user_id JOIN Groupss g ON g.chat_id = m.chat_id GROUP BY c.user_name HAVING COUNT(m.chat_id) <= 2;",
            "SELECT COALESCE(phone_number, 0) AS phone FROM contact WHERE ROUND(phone_number/10000000) = 0912 AND (phone_number%10) =8;",
            "SELECT c.user_name FROM contact c JOIN Users u ON c.user_id = u.user_id JOIN Member m ON m.user_id = u.user_id JOIN channel c1 ON c1.chat_id = m.chat_id GROUP BY c.user_name HAVING COUNT(m.chat_id) = 2;",
            "WITH user_ids AS ( SELECT u.user_id FROM Users u JOIN contact c ON u.user_id = c.user_id WHERE c.user_name IN ('hossein', 'ali')), common_groups AS ( SELECT m.chat_id FROM Member m JOIN user_ids u ON m.user_id = u.user_id GROUP BY m.chat_id HAVING COUNT(DISTINCT m.user_id) = 2 ) SELECT t.message_id, t.content, t.msg_date, t.user_id, t.se_and_re_chat_id AS group_id FROM text_message t JOIN common_groups cg ON t.se_and_re_chat_id = cg.chat_id ORDER BY t.msg_date;",
            "SELECT AVG(LENGTH(t.content)) AS average_characters FROM text_message t JOIN Bot b ON t.se_and_re_chat_id = b.chat_id WHERE b.name = 'first_bot';"
        ]
        
        # for query in queries:
        #     cursor.execute(query)
        #     results = cursor.fetchall()
        #     print(f"\n🔹 نتیجه کوئری:\n{results}")

        def execute_query(query):
            try:
                start_time = time.time()
                cursor.execute(query)
                results = cursor.fetchall()
                elapsed_time = time.time() - start_time
                print(f"\n🔹 نتیجه کوئری:\n{results}")
                print(f"✅ Query executed in {elapsed_time:.4f} seconds")
            except Exception as e:
                print(f"❌ Query failed: {e}")

        # اجرای همزمان کوئری‌ها با چندین ترد
        threads = []
        for query in queries:
            thread = threading.Thread(target=execute_query, args=(query,))
            threads.append(thread)
            thread.start()

        for thread in threads:
            thread.join()

        # بستن اتصال
        cursor.close()
        conn.close()

    except Exception as e:
        print(f"❌ خطا در اجرای SQL: {e}")

# اجرای فایل SQL
execute_sql_file("database_script.sql")


✅ فایل SQL با موفقیت اجرا شد!

🔹 نتیجه کوئری:
[(Decimal('10.500000000000000000'),)]
✅ Query executed in 0.0523 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0523 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0523 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0523 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0513 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0513 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0528 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0528 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0528 seconds

🔹 نتیجه کوئری:
[]
✅ Query executed in 0.0528 seconds
